OpenVINO聲音辨識範例

DevCloud 自帶OpenVINO 2020.3.356(等於openvino-lts)及2021.3.394(等於openvino_2021)版, 操作時依選定環境自動切換。  
本範例須於2021.3.394版下執行。

歐尼克斯實境互動工作室 OmniXRI Jack, 2021.6.15

1.檢查OpenVINO版本

In [1]:
!ls /opt/intel/

intel_sdp_products.db  openvino_2020.3.356  openvino_fpga_2020.3.356
mediasdk	       openvino_2021	    openvino-lts
openvino	       openvino_2021.3.394


2.下載模型  

可選用聲音辨識模型為  
aclnet  
aclnet-int8  

這裡選用 --name aclnet （可自行變更所需模型名稱）  

In [2]:
# 等價命令 !python3 /opt/intel/openvino_2021/deployment_tools/tools/model_downloader/downloader.py --name aclnet
!downloader.py --name aclnet

################|| Downloading aclnet ||################

========== Downloading /home/u75102/My-Notebooks/public/aclnet/aclnet_des_53.onnx
... 100%, 10709 KB, 6975 KB/s, 1 seconds passed



3.模型轉換

如果下載的是Intel Pretrained Model則不需轉換就自帶IR檔(xml,bin)
若是Public Pretrained Model則須進行轉換成IR檔，系統會自動判別。
--name 參數為待轉換模型名稱

In [3]:
# 等價命令 !python3 /opt/intel/openvino_2021/deployment_tools/tools/model_downloader/converter.py --name aclnet
!converter.py --name aclnet

# 檢查模型轉檔後會產生/FP16, FP32不同精度的IR檔(xml, bin)
!ls public/aclnet
!ls public/aclnet/FP32

========== Converting aclnet to IR (FP16)
Conversion command: /usr/bin/python3 -- /opt/intel/openvino/deployment_tools/model_optimizer/mo.py --framework=onnx --data_type=FP16 --output_dir=/home/u75102/My-Notebooks/public/aclnet/FP16 --model_name=aclnet '--input_shape=[1,1,1,16000]' --input=input --output=output --input_model=/home/u75102/My-Notebooks/public/aclnet/aclnet_des_53.onnx

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/u75102/My-Notebooks/public/aclnet/aclnet_des_53.onnx
	- Path for generated IR: 	/home/u75102/My-Notebooks/public/aclnet/FP16
	- IR output name: 	aclnet
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	input
	- Output layers: 	output
	- Input shapes: 	[1,1,1,16000]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess se

為更清楚了解範例程式可支援的模型，可將models.lst及使用方式列出來確認。（此步驟可忽略）

In [4]:
# 列出可支援的模型名稱
!cat /opt/intel/openvino_2021/inference_engine/demos/sound_classification_demo/python/models.lst

# This file can be used with the --list option of the model downloader.
aclnet
aclnet-int8


4.準備測試聲音檔案  

從Github下載測試聲音樣本(青蛙叫聲 1-17970-A-4_Flog.wav, 公雞叫聲 1-27724-A-1_Rooster.wav, 狗叫聲1-30344-A-0_Dog.wav)  
要使用'https://raw.githubusercontent.com/OmniXRI/Colab_DevCloud_OpenVINO_Samples/main/dataset/1-27724-A-1_Rooster.wav'  
取代  'https://github.com/OmniXRI/Colab_DevCloud_OpenVINO_Samples/blob/main/dataset/1-27724-A-1_Rooster.wav'  
資料來源：公開資料集 ESC-50 https://github.com/karolpiczak/ESC-50  

In [5]:
# 下載測試聲音檔
!wget -N https://raw.githubusercontent.com/OmniXRI/Colab_DevCloud_OpenVINO_Samples/main/dataset/1-30344-A-0_Dog.wav
!ls *.wav

--2021-06-14 11:49:43--  https://raw.githubusercontent.com/OmniXRI/Colab_DevCloud_OpenVINO_Samples/main/dataset/1-30344-A-0_Dog.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 441044 (431K) [audio/wav]
Saving to: ‘1-30344-A-0_Dog.wav’

1-30344-A-0_Dog.wav 100%[===================>] 430.71K  --.-KB/s    in 0.03s   

Last-modified header missing -- time-stamps turned off.
2021-06-14 11:49:44 (13.4 MB/s) - ‘1-30344-A-0_Dog.wav’ saved [441044/441044]

1-17970-A-4_Flog.wav  1-27724-A-1_Rooster.wav  1-30344-A-0_Dog.wav


5.進行推論

聲音辨識範例程式 sound_classification_demo.py  

輸入參數：  
-i 輸入聲音檔案(*.wav)  
-m 模型路徑(*.xml)  
--labels 聲音標籤檔(*.txt)，這裡使用OpenVINO內建aclnet_53cl.txt  
-d 推論裝置(在Colab & DevClouud 只能選CPU, DevCloud上欲使用其它裝置須用指派工作方式達成）  
--sample_rate 16000 聲音取樣頻率（預設16KHz)  

最後顯示推論結果，會自動將輸入聲音檔案切成一秒一段再進行辨識。  

In [6]:
# 進行聲音辨識推論
!python3 \
/opt/intel/openvino_2021.3.394/deployment_tools/inference_engine/demos/sound_classification_demo/python/sound_classification_demo.py \
-i 1-30344-A-0_Dog.wav \
-m public/aclnet/FP32/aclnet.xml \
--labels /opt/intel/openvino_2021/deployment_tools/open_model_zoo/data/dataset_classes/aclnet_53cl.txt \
-d CPU --sample_rate 16000

[ INFO ] Creating Inference Engine
[ INFO ] Loading model public/aclnet/FP32/aclnet.xml
[ INFO ] Loading model to the plugin
[ INFO ] Preparing input
[ INFO ] Starting inference
[ INFO ] [0.00-1.00] - 100.00% Dog
[ INFO ] [1.00-2.00] - 100.00% Dog
[ INFO ] [2.00-3.00] - 100.00% Dog
[ INFO ] [3.00-4.00] - 100.00% Dog
[ INFO ] [4.00-5.00] - 100.00% Dog
[ INFO ] Average infer time - 32.5 ms per clip


列出聲音標籤檔內容，共53類，較ESC-50公開資料集多了最後三類。(本步驟可略過)

In [7]:
!cat /opt/intel/openvino_2021/deployment_tools/open_model_zoo/data/dataset_classes/aclnet_53cl.txt

Dog
Rooster
Pig
Cow
Frog
Cat
Hen
Insects (flying)
Sheep
Crow
Rain
Sea waves
Crackling fire
Crickets
Chirping birds
Water drops
Wind
Pouring water
Toilet flush
Thunderstorm
Crying baby
Sneezing
Clapping
Breathing
Coughing
Footsteps
Laughing
Brushing teeth
Snoring
Drinking sipping
Door knock
Mouse click
Keyboard typing
Door wood creaks
Can opening
Washing machine
Vacuum cleaner
Clock alarm
Clock tick
Glass breaking
Helicopter
Chainsaw
Siren
Car horn
Engine
Train
Church bells
Airplane
Fireworks
Hand saw
Gunshot
Crowd
Speech
